In [2]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [1]:
# nbi:hide_in
# Required libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

In [4]:
# nbi:hide_in
def mtQuantizeUniform(aNum,nBits):
    #Uniformly quantize signed fraction aNum with nBits
    #Notes:The overload level of the quantizer should be 1.0    
    if(aNum >= 0):
        s = 0
    else:
        s = 1
        
    if(np.abs(aNum) >= 1):
        code = 2**(nBits-1)-1
    else:
        code = int(((2**(nBits)-1)*np.abs(aNum)+1)/2)

    if(s == 0):
       return int(code)
    else:
       return int(code + 2**(nBits-1))
   
def mtDequantizeUniform(aQuantizedNum,nBits):
    #Uniformly dequantizes nBits-long number aQuantizedNum 
    #into a signed fraction
    if (aQuantizedNum>>(nBits-1)):
        sign = -1
    else:
        sign = 1
    
    if (sign == -1):
        aNum =  sign*float(2*np.abs(aQuantizedNum - 2**(nBits-1)))/(2**nBits -1)    
    else:
        aNum =  sign*float(2*np.abs(aQuantizedNum))/(2**nBits -1) 

    return aNum

In [14]:
nBitsSlider = widgets.IntSlider(
    value=3,
    min=2,
    max=8,
    step=1,
    description='nBits',
    continuous_update=False,
    readout_format='d',
)

zoomSlider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='zoom',
    continuous_update=False,
    readout_format='d',
)


qTypeDropdown = widgets.Dropdown(
    options=['MidTread', 'MidRise'],
    value='MidTread',
    description='Type',
)

In [ ]:
# nbi:hide_in
x = 0.844543

In [15]:
# nbi:hide_in
def plot(nBits, zoom):
    qStep = 2**(-nBits)

    xQ = mtQuantizeUniform(x, nBits)
    y = mtDequantizeUniform(xQ, nBits)
    yAv = np.zeros(2**nBits - 1)
    for i in np.arange(len(yAv)):
        if i == xQ:
            continue
        yAv[i] = mtDequantizeUniform(i, nBits)
        
    zm = 2**zoom
    eb1 = plt.errorbar(0.05, y, yerr=qStep, capsize = 5, color=np.ones(3)*0.5, label='Uncertainty Range')
    eb1[-1][0].set_linestyle(':')
    eb2 = plt.errorbar(-0.05, (y+x)/2, yerr=np.abs(y-x)/2, capsize = 5, color='r', label='Error')
    plt.plot(np.zeros_like(yAv), yAv, 'xb', label='Available Steps')
    plt.plot(0, x, 'kx', label='Analog Value')
    plt.plot(0, y, 'xc', label='Quantized Value')
    plt.legend()
    plt.ylabel('Amlpliutde')
    plt.xlabel(None)
    plt.xticks([0], ["Some Sample"])
    plt.xlim([-1, 1])
    plt.ylim(np.array([-1, 1]) / zm + x)
    plt.grid(True)

widgets.interactive(plot, nBits=nBitsSlider, zoom=zoomSlider)

interactive(children=(IntSlider(value=3, continuous_update=False, description='nBits', max=8, min=2), IntSlide…